In [104]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3,4,5"
from transformers import AutoTokenizer, AutoModel
import torch
# 选择预训练模型，例如BERT
model_name = ['deepseek-ai/DeepSeek-Prover-V1.5-RL','/home/limengze/DeepSeek-Coder/finetune/finetune_result5/checkpoint-40']
tokenizer0 = AutoTokenizer.from_pretrained(model_name[0])
model0 = AutoModel.from_pretrained(model_name[0],device_map='auto')
model0.config.output_hidden_states = True

/home/limengze/miniconda3/envs/ATP_Train/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.58s/it]
We've detected an older driver with an RTX 4000 series GPU. These drivers have issues with P2P. This can affect the multi-gpu inference when using accelerate device_map.Please make sure to update your driver to the latest version which resolves this.


In [126]:
import json
save_pth = 'new_dataset/'
with open(save_pth+'super_dataset.json','r') as f:
    subgoals=json.load(f)

In [106]:
import re
def seg(text):
    split_text = re.split(r'\[\d+\]', text)
    output=split_text[1:]
    return output

In [ ]:

from tqdm import tqdm
j=0
with torch.no_grad():
    for item in tqdm(subgoals):
        # if i<len(outs):
        #     continue
        outs=[]
        for text in seg(item['informal_proof_root']):
            tokens=tokenizer0(text, return_tensors='pt')
            output = model0(**tokens)
            embList=[]
            for i in range(len(output.hidden_states)):
                embeddings = output.hidden_states[i]
                embeddings = embeddings.numpy().mean(axis=1)
                embList.append(embeddings)
            outs.append(embList)
        subgoals[j]['vec']=outs
        j+=1

100%|██████████| 961/961 [04:57<00:00,  3.23it/s]


In [145]:
import copy
backup=copy.deepcopy(subgoals)

In [148]:
with open(save_pth+'super_dataset.pkl','wb') as f:
    pickle.dump(subgoals,f)

In [8]:
import pickle
with open(save_pth+'outs.pkl','wb') as f:
    pickle.dump(outs[0],f)

In [59]:
embList=[]
for i in range(len(outs[0])):
            embeddings = outs[0][i]
            embeddings = embeddings.numpy().mean(axis=1)
            embList.append(embeddings)

In [275]:
#载入做题数据，进行反向翻译
import json
router_pth = 'router_data/'
with open(router_pth+'name_f_c.json','r') as f:
    finetune=json.load(f)
with open(router_pth+'name_ori_c.json','r') as f:
    ori=json.load(f)
with open(router_pth+'name_f_nc.json','r') as f:
    finetune_nc=json.load(f)

In [278]:
#制作倒查表
search_dict={}
for i in range(len(subgoals)):
    search_dict[subgoals[i]['name']]={'detail':subgoals[i],'ori_proofs':[],'fine_proofs':[],'fine_nc_proofs':[]}

In [159]:
def name_translate(name):
    name = name.split('/')[-1]
    true_name = '_'.join(name.split('_')[1:-1])
    set = name.split('_')[-1][1:-1]
    set = set.split(',')
    set = [int(i) for i in set if i!='']
    return true_name,set
name_translate('logs/1121super_cot/135_amc12a_2008_p8_(0,1)')

('amc12a_2008_p8', [0, 1])

In [279]:
#将做题数据和推理数据进行匹配
for i in range(len(fintune)):
    name,set= name_translate(fintune[i])
    search_dict[name]['fine_proofs'].append(set)
for i in range(len(ori)):
    name,set= name_translate(ori[i])
    search_dict[name]['ori_proofs'].append(set)
for i in range(len(finetune_nc)):
    name,set= name_translate(finetune_nc[i])
    search_dict[name]['fine_nc_proofs'].append(set)

In [ ]:
# 统计三个集合中的推理数据
a=0
b=0
c=0
sum=0
for key in search_dict:
    if search_dict[key]['detail']['category']!='minif2f' and 'AIME' in key:
        success=0
        if len(search_dict[key]['fine_proofs'])>0:
            a+=1
            success=1
        if len(search_dict[key]['ori_proofs'])>0:
            b+=1
            success=1
        if len(search_dict[key]['fine_nc_proofs'])>0:
            c+=1
            success=1
        sum+=success
print(a,b,c,sum)

0 0 0 0


In [ ]:
#设置训练集
#并归还向量组
train_set={}
globali=0
for item in search_dict:
    if search_dict[item]['detail']['split']=='test' and search_dict[item]['detail']['category']=='minif2f' and :(len(search_dict[item]['ori_proofs'])!=0 or len(search_dict[item]['fine_proofs'])!=0)
        train_set[item]=search_dict[item]['detail']


In [163]:
# 统计训练集中各个证明的出现次序
def get_order(proofs,n):
    order={}
    sum=0
    for i in range(n):
        order[i]=0
    for proof in proofs:
        for i in range(len(proof)):
            order[proof[i]]+=1
            sum+=1
    #进行归一化
    if sum !=0:
        for i in range(n):
            order[i]=order[i]/sum    
    return order

for item in train_set:
    name = item
    train_set[item]['ori_proofs'] = search_dict[name]['ori_proofs']
    train_set[item]['fine_proofs'] = search_dict[name]['fine_proofs']
    train_set[item]['ori_order']=get_order(search_dict[name]['ori_proofs'],len(search_dict[item]['detail']['informal_proof_subgoals']))

In [165]:
#分离测试集
test_set={}
globali=0
for item in search_dict:
    if search_dict[item]['detail']['category']!='minif2f':
        test_set[item]=search_dict[item]


In [180]:
train_set['mathd_algebra_478']['ori_order'][0]

0.2222222222222222

In [301]:
#把所有训练集的重要度都抽取出来
imp_map=[]
layer=2
for item in train_set:
    ori_order = train_set[item]['ori_order']
    samples= train_set[item]['vec']
    samples = [item2[layer] for item2 in samples]
    for i in ori_order.keys():
        imp_map.append((samples[i],ori_order[i]))


In [402]:
def sim(vec1,vec2):
    df=(vec1-vec2)
    dot = (df*df).sum()
    return -dot**0.5
    # dot = (vec1*vec2).sum()
    # l1 =  (vec1*vec1).sum()**0.5
    # l2 =  (vec2*vec2).sum()**0.5
    return dot/(l1*l2)
def get_most_similar(vec,imp_map):
    sim_values=[]
    for i in range(len(imp_map)):
        simi = sim(vec,imp_map[i][0])
        sim_values.append((i,simi,imp_map[i][1]))
    sim_values=sorted(sim_values,key=lambda x:x[1],reverse=True)
    return sim_values
def test_item(input,imp_map,layer):
    samples=input['detail']['vec']
    samples = [item[layer] for item in samples]
    # 为每个证明计算相似度，找到最相似的证明
    sim_values=[]
    for i in range(len(samples)):
        sim_list = get_most_similar(samples[i],imp_map)
        value = sim_list[0][2]
        sim_values.append((i,value))
    # 排序
    # print(sim_values)
    sim_values=sorted(sim_values,key=lambda x:x[1],reverse=True)
    sum=0
    result=[]
    # result=[sim_values[-1][1]] #把结果进行储存
    # print(sim_values)
    for i in range(len(sim_values)):
        if sum>0.3 or sim_values[i][1]<1e-6:
            break
        sum+=sim_values[i][1]
        result.append(sim_values[i][0])
    # 对result从小到大排序
    result=sorted(result)
    return result

In [324]:
quick_test={}
for item in test_set:
    if (len(test_set[item]['ori_proofs'])!=0 or len(test_set[item]['fine_proofs'])!=0):
        quick_test[item]=test_set[item]

In [403]:
ori_suc=0
fine_suc=0
empty_suc=0
mix_suc=0
j=0
for name in tqdm(quick_test.keys()):
    output=test_item(quick_test[name],imp_map,layer)
    if select_list[j]==0:
        if output in quick_test[name]['ori_proofs']:
            ori_suc+=1
    else:
        if output in quick_test[name]['fine_proofs']:
            fine_suc+=1
    if [] in test_set[name]['ori_proofs']:
        empty_suc+=1
    if output in quick_test[name]['ori_proofs'] or output in quick_test[name]['fine_proofs']:
        mix_suc+=1
    j+=1
print(ori_suc+fine_suc,empty_suc,mix_suc,len(quick_test))

100%|██████████| 84/84 [00:01<00:00, 44.89it/s]

52 53 62 84


In [395]:
for name in tqdm(quick_test.keys()):
    print(quick_test[name]['ori_proofs'])
    print([1,0] in quick_test[name]['ori_proofs'])
    break

  0%|          | 0/84 [00:00<?, ?it/s]

[[1], [2], [5], [0, 1], [0, 2], [0, 3], [0, 5], [1, 2], [1, 3], [1, 4], [1, 5], [2, 4], [2, 5], [3, 4], [3, 5], [4, 5], [0, 1, 2], [0, 1, 3], [0, 1, 4], [0, 2, 3], [0, 2, 4], [0, 3, 4], [0, 3, 5], [0, 4, 5], [1, 2, 3], [1, 2, 4], [1, 3, 4], [1, 3, 5], [1, 4, 5], [2, 3, 4], [2, 3, 5], [2, 4, 5], [3, 4, 5], [0, 1, 2, 3], [0, 1, 2, 4], [0, 1, 3, 4], [0, 1, 3, 5], [0, 1, 4, 5], [0, 2, 3, 4], [0, 2, 3, 5], [0, 2, 4, 5], [0, 3, 4, 5], [1, 2, 3, 4], [1, 2, 3, 5], [1, 2, 4, 5], [1, 3, 4, 5], [2, 3, 4, 5], [0, 1, 2, 3, 4], [0, 1, 2, 3, 5], [0, 1, 2, 4, 5], [0, 1, 3, 4, 5], [0, 2, 3, 4, 5], [1, 2, 3, 4, 5], [0, 1, 2, 3, 4, 5], [1], [2], [5], [0, 1], [0, 2], [0, 3], [0, 5], [1, 2], [1, 3], [1, 4], [1, 5], [2, 4], [2, 5], [3, 4], [3, 5], [4, 5], [0, 1, 2], [0, 1, 3], [0, 1, 4], [0, 2, 3], [0, 2, 4], [0, 3, 4], [0, 3, 5], [0, 4, 5], [1, 2, 3], [1, 2, 4], [1, 3, 4], [1, 3, 5], [1, 4, 5], [2, 3, 4], [2, 3, 5], [2, 4, 5], [3, 4, 5], [0, 1, 2, 3], [0, 1, 2, 4], [0, 1, 3, 4], [0, 1, 3, 5], [0, 1, 4, 5],

68

In [ ]:
test_item(quick_test[name],imp_map,layer)
quick_test[name]['ori_proofs']

从本段开始，构建一个对finetune / ori 的分类路由器

# 打标签
对每个train_set中的证明，计算出它在不同集合之间的隶属度

In [373]:
for item in train_set.keys():
    a=len(search_dict[item]['ori_proofs'])
    b=len(search_dict[item]['fine_proofs'])
    ori_value = a / (a+b)
    fine_value = b / (a+b)
    train_set[item]['ori_value']=ori_value
    train_set[item]['fine_value']=fine_value
    print(a,b)

14 15
0 1
17 25
89 86
1 1
1 0
8 8
0 2
4 3
10 9
6 8
8 7
108 80
58 59
14 13
31 21
0 1
6 6
30 25
2 19
8 7
8 7
8 8
4 5
0 2
17 23
16 14
108 90
16 15
27 29
16 16
15 14
20 21
1 1
12 15
15 16
38 45
15 12
0 6
29 26
59 44
25 19
30 30
15 14
21 17
33 41
8 82
14 15
19 24
32 30
6 7
2 0
30 11
32 29
6 4
15 7
1 2
16 14
48 38
14 14
0 7
1 1
11 7
4 7
21 28
0 5
58 36
19 15
28 30
32 31
17 20
16 16
15 11
7 6
16 28
4 5
1 3
16 16
39 55
252 247
30 16
13 14
5 2
13 14
47 45
2 2
8 5
32 31
27 27
8 7
15 14
57 56
24 23
7 5
63 53
12 12
30 24
16 15
23 30
6 3
7 2
1 1
42 23
61 55
5 6
8 8
1 2
1 2
1 1
89 106
62 55
14 6
7 13
9 10
35 31
64 57
24 22
26 26
5 6
5 7
16 14
28 27
11 19
9 11


In [374]:
for item in quick_test.keys():
    a=len(search_dict[item]['ori_proofs'])
    b=len(search_dict[item]['fine_proofs'])
    ori_value = a / (a+b)
    fine_value = b / (a+b)
    quick_test[item]['ori_value']=ori_value
    quick_test[item]['fine_value']=fine_value
    print(a,b)

54 58
5 5
32 26
62 55
1 1
31 27
11 18
1 1
32 28
32 28
167 84
10 14
8 4
1 2
54 53
3 3
16 15
16 10
3 5
10 9
32 20
289 447
8 11
12 81
1690 1380
3 4
13 14
8 5
15 11
9 21
7 7
7 6
3 14
1 0
12 5
16 13
28 23
26 21
52 144
24 25
405 388
11 12
1 1
1 1
28 24
3 6
2 5
89 41
1 7
102 82
0 1
9 10
0 1
102 118
27 32
1 18
30 15
7 6
21 22
8 8
1 1
12 11
82 135
59 48
1106 1081
1 12
1 1
9 10
1 0
8 11
1 0
25 34
16 10
162 153
1 1
17 19
1 3
35 45
9 6
0 1
22 20
49 40
13 9
10 11


In [ ]:
# 接下来，计算quick_test的分类上限
for item in quick_test.keys():
    a=len(search_dict[item]['ori_proofs'])
    b=len(search_dict[item]['fine_proofs'])
    ori_value = a / (a+b)
    fine_value = b / (a+b)

In [365]:
# 构造cat_map
cat_map=[]

with torch.no_grad():
    for item in tqdm(train_set.keys()):
        # if i<len(outs):
        #     continue
        text = train_set[item]['formal_statement']
        tokens=tokenizer0(text, return_tensors='pt')
        output = model0(**tokens)
        embeddings = output.hidden_states[2]
        embeddings = embeddings.numpy().mean(axis=1)
        cat_map.append((embeddings,train_set[item]['ori_value'],train_set[item]['fine_value']))
    

100%|██████████| 124/124 [00:07<00:00, 16.77it/s]


In [366]:
with torch.no_grad():
        for item in tqdm(quick_test.keys()):
                text = quick_test[item]['detail']['formal_statement']
                tokens=tokenizer0(text, return_tensors='pt')
                output = model0(**tokens)
                embeddings = output.hidden_states[2]
                embeddings = embeddings.numpy().mean(axis=1)
                quick_test[item]['fs_vec']=embeddings

100%|██████████| 84/84 [00:07<00:00, 11.24it/s]


接下来，计算第一次分类的准确性

In [369]:
def sim(vec1,vec2):
    # print(vec1.shape)
    # print(vec2.shape)
    df=(vec1-vec2)
    dot = (df*df).sum()
    return -dot**0.5

def get_most_similar(vec,imp_map):
    sim_values=[]
    for i in range(len(imp_map)):
        simi = sim(vec,imp_map[i][0])
        sim_values.append((i,simi,imp_map[i][1]))
    sim_values=sorted(sim_values,key=lambda x:x[1],reverse=True)
    return sim_values
def test_item(input,imp_map,layer):
    samples=input['fs_vec']
    sim_list = get_most_similar(samples,imp_map)
    value = sim_list[0][2]
    if value > 0.5:
        return 0
    else:
        return 1

In [377]:
ori_suc=0
fine_suc=0
empty_suc=0
mix_suc=0
select_list=[]
for name in tqdm(quick_test.keys()):
    output=test_item(quick_test[name],cat_map,layer)
    select_list.append(output)
    if quick_test[item]['ori_value']>0 and output==0:
        ori_suc+=1
    if quick_test[item]['fine_value']>0 and output==1:
        fine_suc+=1
print(ori_suc,fine_suc,empty_suc,mix_suc,len(quick_test))

100%|██████████| 84/84 [00:00<00:00, 671.91it/s]

45 39 0 0 84


[(tensor([[[-0.9844,  0.7088, -0.3782,  ...,  0.5064, -0.2659,  0.3826],
           [ 0.1429, -0.0192, -0.5838,  ..., -0.0662, -0.0951,  0.0771],
           [-0.4157, -0.0211, -0.5097,  ..., -0.1866, -0.3001,  0.4765],
           ...,
           [ 0.0975,  0.1068, -0.3071,  ...,  0.0239,  0.0419, -0.0096],
           [-0.2277, -0.1495,  0.2199,  ..., -0.3553,  0.3557,  0.2008],
           [ 0.0530,  0.0203, -0.1679,  ..., -0.0960, -0.0366, -0.0109]]]),
  0.4827586206896552,
  0.5172413793103449),
 (tensor([[[-0.9844,  0.7088, -0.3782,  ...,  0.5064, -0.2659,  0.3826],
           [ 0.1429, -0.0192, -0.5838,  ..., -0.0662, -0.0951,  0.0771],
           [-0.0993, -0.1407, -0.1238,  ...,  0.2290,  0.2582,  0.0811],
           ...,
           [ 0.0911,  0.0160, -0.4604,  ...,  0.1215,  0.0258,  0.0805],
           [-0.2669, -0.1322,  0.3126,  ..., -0.3436,  0.3156,  0.1644],
           [-1.0426, -0.4395,  0.1324,  ...,  1.3404, -0.8507,  2.4641]]]),
  0.0,
  1.0),
 (tensor([[[-0.9844,  0.70

In [360]:
quick_test[name]['fs_vec']

tensor([[[-0.9844,  0.7088, -0.3782,  ...,  0.5064, -0.2659,  0.3826],
         [-0.1982, -0.0444, -0.5099,  ..., -0.3076,  0.0794,  0.1470],
         [ 0.0401, -0.0170, -0.2056,  ...,  0.0094, -0.1137,  0.0900],
         ...,
         [-0.0091,  0.0876, -0.2176,  ...,  0.0773, -0.0086, -0.0340],
         [-0.2630, -0.2007,  0.3184,  ..., -0.2928,  0.2537,  0.1275],
         [-0.0708, -0.0537,  0.3966,  ..., -0.1662,  0.0589,  0.0626]]])